In [1]:
import requests
import pandas as pd
import json
import time
import re
import numpy as np
import spacy
import psycopg2

In [23]:
info_vac_v1 = pd.DataFrame()

for i in range(0,19):
    
    params = {
        'text': '!(аналитик данных OR data analyst OR бизнес-аналитик OR BI-аналитик or data engineer)', # Текст фильтра
        'area': 1, # Поиск ощуществляется по вакансиям города Москва
        'page': i, # Индекс страницы поиска на HH
        'per_page': 100 # Кол-во вакансий на 1 странице
        }
    
    req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
    data = req.content.decode() # Декодируем его ответ, чтобы Кириллица отображалась корректно
    time.sleep(0.5)
    req.close()
    json_data = json.loads(data)
    new_data = pd.DataFrame(json_data['items']).reset_index(drop=True)
    info_vac_v1 = pd.concat([info_vac_v1, new_data], ignore_index=True)

In [24]:

salary = pd.json_normalize(info_vac_v1['salary'])
address = pd.json_normalize(info_vac_v1['address'])

address = address.drop(columns=['id'])

employer = pd.json_normalize(info_vac_v1['employer'])
employer = employer.drop(columns=['logo_urls.90', 'logo_urls.240', 'url', 'alternate_url', 'vacancies_url', 'logo_urls.original', 'logo_urls' ])
employer = employer.rename(columns={'id': 'id_employer', "name": "name_employer"})

snippet = pd.json_normalize(info_vac_v1['snippet'])

schedule = pd.json_normalize(info_vac_v1['schedule'])
schedule.drop(columns=['id'], inplace=True)
schedule.rename(columns={'name': 'schedules'}, inplace=True)

def sep_list_columns(list):
    length = len(list)
    if length >= 1:
        return list[0]

work_format = pd.json_normalize(info_vac_v1['work_format'].apply(sep_list_columns))
work_format.rename(columns={'name': 'format_works'}, inplace=True)

working_hours = pd.json_normalize(info_vac_v1['working_hours'].apply(sep_list_columns))
working_hours.rename(columns={'name': 'hours_working'}, inplace=True)

work_schedule_by_days = pd.json_normalize(info_vac_v1['work_schedule_by_days'].apply(sep_list_columns))
work_schedule_by_days.rename(columns={'name': 'schedule_work_by_days'}, inplace=True)

professional_roles = pd.json_normalize(info_vac_v1['professional_roles'].apply(sep_list_columns))
professional_roles.rename(columns={'name': 'roles_professional'}, inplace=True)

experience = pd.json_normalize(info_vac_v1['experience'])
experience = experience.drop(columns=['id'])
experience = experience.rename(columns={'name': 'experience_'})

employment = pd.json_normalize(info_vac_v1['employment'])
employment = employment.drop(columns=['id'])
employment = employment.rename(columns={'name': 'employment_'})

info_vac = pd.concat([info_vac_v1, salary, address, employer, snippet, schedule, work_format['format_works'], working_hours['hours_working'], work_schedule_by_days['schedule_work_by_days'], professional_roles['roles_professional'], experience['experience_'], employment['employment_']], axis = 1)
info_vac = info_vac.drop(
    columns=['area', 'salary', 'employment_form', 'salary_range', 'type', 'address', 'employer', 'contacts', 'snippet',
             'schedule', 'work_format', 'employment_form', 'working_hours', 'work_schedule_by_days',
             'professional_roles', 'experience', 'employment', 'metro_stations', 'branding', 'department', 'description', 'insider_interview', 'response_url', 'sort_point_distance','adv_response_url','adv_context','metro','relations', 'working_days',
             'working_time_intervals', 'working_time_modes', 'fly_in_fly_out_duration', 'night_shifts', 'is_adv_vacancy', 'archived', 'show_logo_in_search', 'show_contacts', 'video_vacancy', 'brand_snippet', 'employer_rating'
             ])

exchange_rates = {
    'USD': 80.0,  # 1 USD = 80 RUB
    'EUR': 90.0,  # 1 EUR = 90 RUB
    'RUR': 1.0     # 1 RUB = 1 RUB
}

def check_salary(row):
    
    if pd.isna(row['to']) and pd.isna(row['from']):
        return np.nan
    elif pd.isna(row['to']):
        total_salary = row['from']
    elif pd.isna(row['from']):
        total_salary = row['to']
    else:
        total_salary = (row['from'] + row['to']) / 2
    
    currency = row['currency']
    if pd.notna(currency):
        total_salary *= exchange_rates.get(currency, 1.0) # Конвертация в рубли
    if pd.notna(row['gross']) and row['gross'] == True:
        total_salary *= 0.87 # Вычет налогов
    return total_salary

info_vac['total_salary'] = info_vac.apply(check_salary, axis = 1)

In [27]:
description_info = []
for id in info_vac['id']:
    try:
        # Делаем запрос к API
        req_vac = requests.get(f'https://api.hh.ru/vacancies/{id}') 
        req_vac.raise_for_status() 
        data = req_vac.content.decode() 
        req_vac.close()
        
        time.sleep(0.4)  # Задержка между запросами
        
        data_vac = json.loads(data)
        
        # Обработка skills с проверкой наличия ключа
        skills = []
        if 'key_skills' in data_vac and isinstance(data_vac['key_skills'], list):
            skills = [i['name'] for i in data_vac['key_skills']]
        
        description_info.append({
            'vacancy_id': id,
            'description': data_vac.get('description', ''), 
            'key_skills_under_desc': skills,
        })
    except Exception as e:
        print(f"Неожиданная ошибка для вакансии {id}: {e}")
        continue
        

In [28]:
df_description_info = pd.DataFrame(description_info)
info_vac_merge = info_vac.merge(df_description_info, left_on='id', right_on='vacancy_id', how='left').drop(columns = ['vacancy_id'])

info_vac_merge.rename(columns={'description_y': 'description',
                         'key_skills_under_desc_y': 'key_skills_under_desc'}, inplace=True)
info_vac_merge.to_csv('info_vac.csv')

In [30]:
info_vac_merge

,id,premium,name,has_test,response_letter_required,published_at,created_at,apply_alternate_url,url,alternate_url,...,schedules,format_works,hours_working,schedule_work_by_days,roles_professional,experience_,employment_,total_salary,description,key_skills_under_desc
0,122701655,False,Младший аналитик,False,False,2025-07-11T11:29:10+0300,2025-07-11T11:29:10+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/122701655?host=hh.ru,https://hh.ru/vacancy/122701655,...,Полный день,NaN,8 часов,5/2,Аналитик,От 1 года до 3 лет,Полная занятость,NaN,<p><strong>Чем тебе предстоит заниматься:</str...,"[SQL, Python, Визуализация данных]"
1,122617440,False,Аналитик (начинающий специалист),False,False,2025-07-10T10:56:37+0300,2025-07-10T10:56:37+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/122617440?host=hh.ru,https://hh.ru/vacancy/122617440,...,Полный день,Гибрид,8 часов,5/2,Аналитик,Нет опыта,Полная занятость,NaN,<p>Мы предлагаем уникальную возможность для вы...,"[Английский язык, Аналитическое мышление, Стра..."
2,122528622,False,Младший аналитик данных,False,False,2025-07-08T13:40:52+0300,2025-07-08T13:40:52+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/122528622?host=hh.ru,https://hh.ru/vacancy/122528622,...,Удаленная работа,Удалённо,8 часов,5/2,"BI-аналитик, аналитик данных",Нет опыта,Полная занятость,69600.0,<p>Ты только начинаешь свой путь в мире данных...,[]
3,122720013,False,Дата аналитик,False,False,2025-07-11T16:44:28+0300,2025-07-11T16:44:28+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/122720013?host=hh.ru,https://hh.ru/vacancy/122720013,...,Полный день,Гибрид,8 часов,5/2,"BI-аналитик, аналитик данных",От 1 года до 3 лет,Полная занятость,NaN,<strong>Обязанности:</strong> <ul> <li> <p>раб...,"[Аналитическое мышление, Исследовательский ана..."
4,122016069,False,Аналитик сервиса (м.Фили / м.Шелепиха),False,False,2025-06-24T15:53:47+0300,2025-06-24T15:53:47+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/122016069?host=hh.ru,https://hh.ru/vacancy/122016069,...,Полный день,NaN,8 часов,5/2,Другое,От 1 года до 3 лет,Полная занятость,130500.0,<p><strong>«МОСАВТОСАНТРАНС»</strong> - одно и...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1901,118961722,False,Middle Analyst / Аналитик,False,False,2025-06-24T18:46:28+0300,2025-06-24T18:46:28+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/118961722?host=hh.ru,https://hh.ru/vacancy/118961722,...,Полный день,NaN,8 часов,5/2,Аналитик,От 1 года до 3 лет,Полная занятость,NaN,"<p>Привет, мы команда быстрой бизнес-аналитики...",[]
1902,120991772,False,Аналитик отдела продаж,False,False,2025-06-25T07:59:14+0300,2025-06-25T07:59:14+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/120991772?host=hh.ru,https://hh.ru/vacancy/120991772,...,Полный день,На месте работодателя,8 часов,5/2,Аналитик,От 1 года до 3 лет,Полная занятость,NaN,<strong>Обязанности:</strong> <ul> <li>Подгото...,"[MS Excel, Power Pivot, Power Query]"
1903,121871494,False,Аналитик (Систем управления АРМ),False,False,2025-06-20T09:48:04+0300,2025-06-20T09:48:04+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/121871494?host=hh.ru,https://hh.ru/vacancy/121871494,...,Полный день,NaN,8 часов,5/2,Специалист технической поддержки,От 1 года до 3 лет,Полная занятость,NaN,<p>Мы — команда сервиса Систем управления АРМ....,[]
1904,121895577,False,Аналитик по стратегии,False,False,2025-06-20T15:56:33+0300,2025-06-20T15:56:33+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/121895577?host=hh.ru,https://hh.ru/vacancy/121895577,...,Полный день,Гибрид,8 часов,5/2,Аналитик,От 1 года до 3 лет,Полная занятость,NaN,<p><strong>Группа компаний «Эталон»</strong> —...,"[MS PowerPoint, MS Excel, Подготовка

In [29]:
skills_keywords = [
    # Языки программирования
    'SQL', 'Python', 'R', 'Scala', 'Java', 'C#', 'Julia', '1C',
    
    # BI и визуализация
    'Power BI', 'Tableau', 'Qlik', 'Looker', 'Metabase', 'Redash', 'Superset',
    'Excel', 'Google Sheets', 'Data Studio', 'Plotly', 'Matplotlib', 'Seaborn',
    
    # Базы данных
    'PostgreSQL', 'MySQL', 'MS SQL', 'Oracle', 'ClickHouse', 'Greenplum',
    'MongoDB', 'Redis', 'Cassandra', 'Snowflake', 'BigQuery', 'Redshift',
    
    # Big Data
    'Hadoop', 'Spark', 'Hive', 'Kafka', 'Airflow', 'Flink', 'Databricks',
    
    # Облачные платформы
    'AWS', 'Azure', 'GCP', 'Yandex Cloud', 'IBM Cloud',
    
    # ETL и обработка данных
    'DWH', 'ETL', 'ELT', 'Data Vault', 'Data Lake', 'Data Mesh',
    'Informatica', 'Talend', 'SSIS', 'Alteryx', 'dbt', 'Apache NiFi',
    
    # Аналитика
    'Machine Learning', 'ML', 'AI', 'Deep Learning', 'NLP', 'Computer Vision',
    'Statistics', 'A/B тестирование', 'Predictive Modeling', 'Time Series',
    'EDA', 'Feature Engineering', 'MLflow', 'Kubeflow',
    
    # Управление
    'Scrum', 'Agile', 'Kanban', 'Jira', 'Confluence', 'Git', 'CI/CD',
    
    # Дополнительные технологии
    'Docker', 'Kubernetes', 'Linux', 'Bash', 'Pandas', 'NumPy', 'SciPy',
    'Scikit-learn', 'TensorFlow', 'PyTorch', 'Keras', 'XGBoost', 'CatBoost',
    'LightGBM', 'OpenCV', 'NLTK', 'spaCy', 'Hugging Face',]

def extract_skills(desc):
    found_skills = []
    desc = str(desc).lower()
    for skill in skills_keywords:
        if re.search(r'\b' + re.escape(skill.lower()) + r'\b', desc):
            found_skills.append(skill)
    return found_skills

nlp = spacy.load("ru_core_news_sm")
def check_intern_nlp(desc):
    
    doc = nlp(desc.lower()) 

    internship_lemmas = {
        "интерн", "стажёр", 
        "intern", "trainee" 
    }
    for token in doc:
        # Проверяем лемму (начальную форму) слова
        if token.lemma_ in internship_lemmas:
            return True
            
    return False

info_vac_merge = pd.read_csv('info_vac.csv', index_col=0)

info_vac_merge.columns = map(lambda x: x.replace('.', '_'), info_vac_merge.columns.to_list())

info_vac_merge['skills_in_desc'] = info_vac_merge['description'].apply(extract_skills)
info_vac_merge = info_vac_merge[info_vac_merge['id'].notna()]
info_vac_merge['is_intern'] = info_vac_merge['name'].apply(check_intern_nlp)

skills_desc = info_vac_merge['skills_in_desc'].explode('skills_in_desc')
skills_counts = skills_desc.value_counts()
vacancy_skills = info_vac_merge[['id', 'skills_in_desc']].explode('skills_in_desc')

employer = info_vac_merge[['id_employer', 'name_employer', 'accredited_it_employer', 'trusted', 'employer_rating.total_rating', 'employer_rating.reviews_count']].drop_duplicates(subset = ['id_employer'], keep = 'first')
employer = employer[employer['id_employer'].notna()]
info_vac_merge.drop(columns=['name_employer', 'accredited_it_employer', 'trusted', 'employer_rating.total_rating', 'employer_rating.reviews_count'], inplace=True)

In [33]:
info_vac.to_csv('info_about_vacancies.csv')
skills_counts.to_csv('skills_counts.csv')
vacancy_skills.to_csv('vacancy_skills.csv')

In [33]:


DB_CONFIG = {
    'host': 'localhost',
    'port': '5433',
    'database': 'postgres',
    'user': 'postgres',
    'password': '123'
}

with psycopg2.connect(**DB_CONFIG) as conn:
    with conn.cursor() as cursor:

create_table_query = """
  CREATE TABLE IF NOT EXISTS employers (
      id_employer INTEGER PRIMARY KEY,
      name_employer VARCHAR(200),
      accredited_it_employer BOOLEAN,
      trusted BOOLEAN,
      total_rating NUMERIC(10, 2),
      reviews_count NUMERIC(10,2));
  """

cursor.execute(create_table_query)
conn.commit()

columns = ', '.join(employer.columns)
placeholders = ', '.join(['%s'] * len(employer.columns))
with conn.cursor() as cursor:
    for _, row in employer.iterrows():
        cursor.execute(f"INSERT INTO employers ({columns}) "
                       f"VALUES ({placeholders})"
                       f"ON CONFLICT (id_employer) DO NOTHING", tuple(row))
        conn.commit()



In [32]:
info_vac_merge

,id,premium,name,has_test,response_letter_required,published_at,created_at,apply_alternate_url,url,alternate_url,...,hours_working,schedule_work_by_days,roles_professional,experience_,employment_,total_salary,description,key_skills_under_desc,skills_in_desc,is_intern
0,122701655,False,Младший аналитик,False,False,2025-07-11T11:29:10+0300,2025-07-11T11:29:10+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/122701655?host=hh.ru,https://hh.ru/vacancy/122701655,...,8 часов,5/2,Аналитик,От 1 года до 3 лет,Полная занятость,NaN,<p><strong>Чем тебе предстоит заниматься:</str...,"['SQL', 'Python', 'Визуализация данных']","[SQL, Power BI, Excel, ClickHouse, Airflow, DWH]",False
1,122617440,False,Аналитик (начинающий специалист),False,False,2025-07-10T10:56:37+0300,2025-07-10T10:56:37+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/122617440?host=hh.ru,https://hh.ru/vacancy/122617440,...,8 часов,5/2,Аналитик,Нет опыта,Полная занятость,NaN,<p>Мы предлагаем уникальную возможность для вы...,"['Английский язык', 'Аналитическое мышление', ...","[Power BI, Excel]",False
2,122528622,False,Младший аналитик данных,False,False,2025-07-08T13:40:52+0300,2025-07-08T13:40:52+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/122528622?host=hh.ru,https://hh.ru/vacancy/122528622,...,8 часов,5/2,"BI-аналитик, аналитик данных",Нет опыта,Полная занятость,69600.0,<p>Ты только начинаешь свой путь в мире данных...,[],"[SQL, Python, ML, EDA]",False
3,122720013,False,Дата аналитик,False,False,2025-07-11T16:44:28+0300,2025-07-11T16:44:28+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/122720013?host=hh.ru,https://hh.ru/vacancy/122720013,...,8 часов,5/2,"BI-аналитик, аналитик данных",От 1 года до 3 лет,Полная занятость,NaN,<strong>Обязанности:</strong> <ul> <li> <p>раб...,"['Аналитическое мышление', 'Исследовательский ...","[SQL, Python, Power BI, Tableau, Matplotlib, S...",False
4,122016069,False,Аналитик сервиса (м.Фили / м.Шелепиха),False,False,2025-06-24T15:53:47+0300,2025-06-24T15:53:47+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/122016069?host=hh.ru,https://hh.ru/vacancy/122016069,...,8 часов,5/2,Другое,От 1 года до 3 лет,Полная занятость,130500.0,<p><strong>«МОСАВТОСАНТРАНС»</strong> - одно и...,[],[Excel],False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1901,118961722,False,Middle Analyst / Аналитик,False,False,2025-06-24T18:46:28+0300,2025-06-24T18:46:28+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/118961722?host=hh.ru,https://hh.ru/vacancy/118961722,...,8 часов,5/2,Аналитик,От 1 года до 3 лет,Полная занятость,NaN,"<p>Привет, мы команда быстрой бизнес-аналитики...",[],"[Python, R, Excel]",False
1902,120991772,False,Аналитик отдела продаж,False,False,2025-06-25T07:59:14+0300,2025-06-25T07:59:14+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/120991772?host=hh.ru,https://hh.ru/vacancy/120991772,...,8 часов,5/2,Аналитик,От 1 года до 3 лет,Полная занятость,NaN,<strong>Обязанности:</strong> <ul> <li>Подгото...,"['MS Excel', 'Power Pivot', 'Power Query']",[Excel],False
1903,121871494,False,Аналитик (Систем управления АРМ),False,False,2025-06-20T09:48:04+0300,2025-06-20T09:48:04+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/121871494?host=hh.ru,https://hh.ru/vacancy/121871494,...,8 часов,5/2,Специалист технической поддержки,От 1 года до 3 лет,Полная занятость,NaN,<p>Мы — команда сервиса Систем управления АРМ....,[],"[Jira, Confluence, Linux]",False
1904,121895577,False,Аналитик по стратегии,False,False,2025-06-20T15:56:33+0300,2025-06-20T15:56:33+0300,https://hh.ru/applicant/vacancy_response?vacan...,https://api.hh.ru/vacancies/121895577?host=hh.ru,https://hh.ru/vacancy/121895577,...,8 часов,5/2,Аналитик,От 1 года до 3 лет,Полная занятос

In [38]:
columns

'id_employer, name_employer, accredited_it_employer, trusted, employer_rating.total_rating, employer_rating.reviews_count'

In [6]:
employer

,id_employer,name_employer,accredited_it_employer,trusted,total_rating,reviews_count
0,2460946.0,Самокат (ООО Умный ритейл),False,True,3.8,2699.0
1,3984.0,Рексофт,True,True,4.5,58.0
2,5995606.0,Панина Анастасия Александровна,False,True,NaN,NaN
3,8582.0,Лемана ПРО,False,True,4.0,6271.0
4,3365154.0,ГУП Автокомбинат Мосавтосантранс,False,True,3.8,119.0
...,...,...,...,...,...,...
1894,2545830.0,ЛОТ-Золото,False,True,NaN,NaN
1900,9678069.0,Текс-Мод,False,True,NaN,NaN
1902,5384634.0,Норебо Холдинг,False,True,NaN,NaN
1904,140987.0,Группа компаний Эталон,False,True,NaN,NaN
